# Explore here

It's recommended to use this notebook for exploration purposes.

In [ ]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

resource_url = "https://en.wikipedia.org/wiki/List_of_Spotify_streaming_records"

response = requests.get(resource_url, time.sleep(10))
if response:
    soup = BeautifulSoup(response.text, 'html')
    soup
data = []
# Find the table
table = soup.find('table', class_='wikitable')
# Find the header
header = table.find('tr')
# Find the header columns
header_row = header.find('th')
# We loop through them and print them in text
for th in header_row:
    title = th.text.strip()

['Rank']
